# Overall: A Statistic 

My intent is to create a new statistic that summarizes a player's total ability on the basketball court. Similar to player ratings in 2k, giving a quick glance at this stat should provide basic insight as to how good a player is  currently. This stat will be composed of numerous known box score and advanced stats. It will be designed by learning from machine learning models to decide the importance (or weight) to assign each sub-stat that makes up the overall rating. The sub-stats will ultimately combine to form an overall player rating. 

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns 
pd.set_option('display.max_columns', None)
import tensorflow as tf, keras 
from sklearn.linear_model import LinearRegression, LogisticRegression 
import sqlalchemy 

## Concept of the Overall Rating 

Goal 
- capture player ability with a single statistic 

What is Player Ability? 
- the ability to generate positive output on the court 
- impact the game offensively and defensively 
- ability to stay available? potential statistical adjustment 

Problem Type 
- prediction problem for discrete variable 
- output a continuous variable; convert to discrete 

Data Sources 
- all-star dataset 
- accolade data? 

Machine Learning Process 
- unsupervised learning task  
- clustering->list of ratings->rating 
- percentile = rating? 
- compare output to historical observations to generate percentile 
- normalize all stats relative to eras 


## Dataset 

In [17]:
engine = sqlalchemy.create_engine('sqlite:///../../DB/ballbase.db') 
df = pd.read_sql('master_as', con=engine).drop('AS', axis=1) 
df 

,Player,Season,Pos,Age,Tm,GP,MPG,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORPG,DRPG,RPG,APG,SPG,BPG,ToPG,PF,PPG,T2P,T2PA,T3P,T3PA,TAST,TBLK,TDRB,TFG,TFGA,TFT,TFTA,TMP,TORB,TPF,TPTS,TSTL,TTOV,TTRB,DRtg,ORtg,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48
0,Abdul Jeelani,1979-80,SF,25.0,POR,77.0,16.7,3.7,7.3,51.0,0.0,0.1,0.0,3.7,7.3,51.5,51.0,2.1,2.6,78.9,1.5,2.0,3.5,1.2,0.5,0.5,1.5,2.0,9.6,288.0,559.0,0.0,6.0,95.0,40.0,156.0,288.0,565.0,161.0,204.0,1286.0,114.0,155.0,737.0,40.0,117.0,270.0,104.0,109.0,0.011,11.6,1.8,1.5,-0.1,13.5,1.5,0.361,1.7,9.6,2.3,18.7,1.5,15.2,11.5,56.3,24.6,1.1,3.8,0.141
1,Adrian Dantley,1979-80,SF,24.0,UTA,68.0,39.3,10.7,18.6,57.6,0.0,0.0,0.0,10.7,18.6,57.7,57.6,6.5,7.7,84.2,2.7,4.9,7.6,2.8,1.4,0.2,3.4,3.1,28.0,730.0,1265.0,0.0,2.0,191.0,14.0,333.0,730.0,1267.0,443.0,526.0,2674.0,183.0,211.0,1903.0,96.0,233.0,516.0,110.0,119.0,0.002,12.3,0.3,3.7,-1.6,14.0,1.1,0.415,5.3,8.3,9.5,24.3,1.8,13.5,11.3,63.5,27.8,3.8,10.5,0.189
2,Al Skinner,1979-80,SG,27.0,PHI,2.0,5.0,0.5,1.0,50.0,0.0,0.0,0.0,0.5,1.0,50.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.5,1.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,10.0,0.0,1.0,2.0,0.0,2.0,0.0,111.0,65.0,0.000,25.4,0.0,-4.8,-0.2,0.0,0.0,0.000,-4.6,0.0,0.0,-0.6,0.0,50.0,0.0,50.0,16.0,0.0,0.0,-0.152
3,Alex English,1979-80,SF,26.0,IND-DEN,78.0,30.8,7.1,14.3,49.7,0.0,0.1,33.3,7.1,14.2,49.8,49.8,2.7,3.4,78.9,3.4,4.3,7.8,2.9,0.9,0.8,2.7,2.6,16.9,551.0,1107.0,2.0,6.0,224.0,62.0,336.0,553.0,1113.0,210.0,266.0,2401.0,269.0,206.0,1318.0,73.0,214.0,605.0,107.0,106.0,0.005,13.9,1.4,0.4,-1.2,14.9,2.0,0.239,1.6,11.3,3.3,18.0,1.4,14.8,13.1,53.6,23.4,1.5,5.3,0.105
4,Allan Bristow,1979-80,SF,28.0,UTA,82.0,28.1,4.6,9.6,48.0,0.0,0.1,28.6,4.6,9.5,48.2,48.2,2.4,3.0,81.1,2.1,4.2,6.2,4.2,1.1,0.1,2.2,2.6,11.6,375.0,778.0,2.0,7.0,341.0,6.0,342.0,377.0,785.0,197.0,243.0,2304.0,170.0,211.0,953.0,88.0,179.0,512.0,110.0,108.0,0.009,21.4,0.1,0.6,-0.8,16.7,1.2,0.310,1.4,9.0,3.2,16.6,1.9,16.7,13.0,53.4,20.0,1.5,4.4,0.091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19370,Zach LaVine,2023-24,SG,28.0,CHI,25.0,34.9,6.8,15.0,45.2,2.4,6.8,34.9,4.4,8.3,53.6,53.1,3.5,4.1,85.4,0.3,4.8,5.2,3.9,0.8,0.3,2.1,2.3,19.5,111.0,207.0,59.0,169.0,98.0,8.0,121.0,170.0,376.0,88.0,103.0,872.0,8.0,57.0,487.0,21.0,52.0,129.0,118.0,113.0,0.449,16.8,0.9,0.1,-0.7,15.8,0.6,0.274,0.8,1.0,0.8,15.1,1.2,11.0,8.3,57.8,23.8,0.5,1.5,0.080
19371,Zavier Simpson,2023-24,PG,26.0,MEM,7.0,23.0,2.4,7.7,31.5,0.7,2.4,29.4,1.7,5.3,32.4,36.1,0.4,0.6,75.0,0.6,2.3,2.9,3.6,1.0,0.4,1.4,1.6,6.0,12.0,37.0,5.0,17.0,25.0,3.0,16.0,17.0,54.0,3.0,4.0,161.0,4.0,11.0,42.0,7.0,10.0,20.0,115.0,86.0,0.315,22.5,1.7,-5.9,0.3,11.3,0.2,0.074,-6.2,2.6,-0.3,6.3,2.1,15.2,6.8,37.7,17.5,-0.2,-0.2,-0.046
19372,Zeke Nnaji,2023-24,PF,23.0,DEN,58.0,9.9,1.2,2.6,46.3,0.1,0.4,26.1,1.1,2.2,50.0,48.3,0.7,1.1,67.7,1.1,1.1,2.2,0.6,0.3,0.7,0.5,1.4,3.2,63.0,126.0,6.0,23.0,32.0,38.0,65.0,69.0,149.0,42.0,62.0,576.0,61.0,81.0,186.0,15.0,27.0,126.0,112.0,113.0,0.154,7.0,5.6,-3.7,0.1,12.2,0.8,0.416,-3.8,12.1,0.4,12.4,1.3,13.3,12.1,52.8,15.4,-0.2,1.2,0.099
19373,Ziaire Williams,2023-24,SF,22.0,MEM,51.0,20.4,2.9,7.4,39.7,1.1,3.7,30.7,1.8,3.7,48.7,47.4,1.2,1.5,82.7,0.7,2.8,3.5,1.5,0.7,0.2,1.3,1.7,8.2,92.0,189.0,58.0,189.0,75.0,9.0,145.0,150.0,378.0,62.0,75.0,1038.0,35.0,85.0,420.0,36.0,66.0,180.0,115.0,98.0,0.500,11.1,0.8,-4.5,-0.4,15.9,1.1,0.198,-4.0,3.5,-0.9,9.3,1.7,13.8,9.5,51.1,19.7,-0.6,0.2,0.010
